In [257]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [258]:
#baseurl = 'https://radiant-island-80986.herokuapp.com/'
baseurl = 'http://127.0.0.1:5000/'

In [259]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [260]:
response.json()

{'signingKey_hex': 'e9bea8d5307a41babfbed0c2f65bcfc86a895a6838ed47642cf7ae69d04a5aab',
 'traderId': '8',
 'verifyKey_hex': '5e6fa66ec70fed45e183de3bc51520d00341209642d705c610159be1f6af3d0a'}

In [261]:
url = baseurl + 'createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [262]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [263]:
pd.DataFrame(response.json(), index=[0])


checks  marketBranchId  marketMax  marketMin  marketRootId traderId
0  False               1          1          0             1        8

In [264]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [265]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [266]:
# View json response
pd.DataFrame(response.json(), index=[0])

checks  marketBranchId  marketRootId  price  quantity traderId
0  (True, True)               1             1  0.551         1        8

In [267]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [268]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '8'}

In [269]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 2.219069242477417 seconds.
matched trade: 1 in 2.2619259357452393 seconds.
matched trade: 2 in 2.1026389598846436 seconds.
matched trade: 3 in 2.1256556510925293 seconds.
matched trade: 4 in 2.1398508548736572 seconds.
matched trade: 5 in 2.1392641067504883 seconds.
matched trade: 6 in 2.188161849975586 seconds.
matched trade: 7 in 2.178122043609619 seconds.
matched trade: 8 in 2.2019240856170654 seconds.
matched trade: 9 in 2.2032268047332764 seconds.


In [270]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [271]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 8}

In [272]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [273]:
a = pd.read_json(response.json())
a

marketRootId  marketBranchId  marketMin  marketMax
0             1               1          0        0.7
1             2               1          0        0.7
2             3               1          0        0.7
3             4               1          0        0.7

In [274]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [275]:
a = pd.read_json(response.json())
a

marketRootId  marketBranchId  price  quantity  traderId
0               1               1  0.551         1         5
1               1               1  0.551        -1         5
10              1               1  0.551         1         5
100             1               1  0.551        -1         6
101             1               1  0.551         1         6
102             1               1  0.551        -1         6
103             1               1  0.551         1         6
104             1               1  0.551         1         6
105             1               1  0.551        -1         6
106             1               1  0.551        -1         6
107             1               1  0.551         1         6
108             1               1  0.551        -1         6
109             1               1  0.551         1         6
11              1               1  0.551        -1         5
110             1               1  0.551         1         6
111             1               1  0.551        -1         6
112             1               1  0.551        -1         6
113             1               1  0.551         1         6
114             1               1  0.551        -1         6
115             1               1  0.551         1         6
116             1               1  0.551         1         6
117             1               1  0.551        -1         6
118             1               1  0.551        -1         6
119             1               1  0.551         1         6
12              1               1  0.551        -1         5
120             1               1  0.551        -1         6
121             1               1  0.551         1         6
122             1               1  0.551         1         6
123             1               1  0.551        -1         6
124             1               1  0.551        -1         6
..            ...             ...    ...       ...       ...
72              1               1  0.440         1         5
73              1               1  0.440         1         5
74              1               1  0.440         1         5
75              1               1  0.440         1         5
76              1               1  0.551         1         6
77              1               1  0.551        -1         6
78              1               1  0.551        -1         6
79              1               1  0.551         1         6
8               1               1  0.551        -1         5
80              1               1  0.551         1         6
81              1               1  0.551        -1         6
82              1               1  0.551        -1         6
83              1               1  0.551         1         6
84              1               1  0.551        -1         6
85              1               1  0.551         1         6
86              1               1  0.551         1         6
87              1               1  0.551        -1         6
88              1               1  0.551        -1         6
89              1               1  0.551         1         6
9               1               1  0.551         1         5
90              1               1  0.551        -1         6
91              1               1  0.551         1         6
92              1               1  0.551         1         6
93              1               1  0.551        -1         6
94              1               1  0.551        -1         6
95              1               1  0.551         1         6
96              1               1  0.551        -1         6
97              1               1  0.551         1         6
98              1               1  0.551         1         6
99              1               1  0.551        -1         6

[294 rows x 5 columns]

In [276]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.0856420993804932 seconds.
matched trade: 1 in 1.0887138843536377 seconds.
matched trade: 2 in 1.0576870441436768 seconds.
matched trade: 3 in 1.0546550750732422 seconds.
matched trade: 4 in 1.0400469303131104 seconds.
matched trade: 5 in 1.0507619380950928 seconds.
matched trade: 6 in 1.0488760471343994 seconds.
matched trade: 7 in 1.045140027999878 seconds.
matched trade: 8 in 1.1173770427703857 seconds.
matched trade: 9 in 1.2807798385620117 seconds.


In [277]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [278]:
pd.read_json(response.json())

marketRootId  marketBranchId  price  quantity  traderId
0             1               1   0.44        10         5
1             1               1   0.44        10         6
2             1               1   0.44        10         7
3             1               1   0.44        10         8

In [279]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [280]:
pd.read_json(response.json())

marketRootId  marketBranchId  price  quantity  traderId
0             1               1  0.551         0         5
1             1               1  0.551         0         6
2             1               1  0.551         0         7
3             1               1  0.551         0         8

In [281]:
tId

'8'